# Music Generation

In [182]:
import pyaudio
import math
from numpy import linspace,sin,pi,int16,rint,sign,arcsin,tan,arctan,cos,append,multiply,add,subtract,divide,repeat,random,clip,fft,zeros,cumsum,minimum
import time
import pandas as pd
from enum import Enum
from scipy.io import wavfile
from scipy.signal import stft,istft

In [3]:
%matplotlib inline

In [ ]:
p = pyaudio.PyAudio()

In [5]:
for i in range(p.get_device_count()):
  dev = p.get_device_info_by_index(i)
  print((i,dev['name'],dev['maxInputChannels']))

(0, 'Microsoft Sound Mapper - Output', 0)
(1, 'Headphones (High Definition Aud', 0)
(2, 'BenQ BL3200 (NVIDIA High Defini', 0)
(3, 'Speakers (2- HTC Vive)', 0)


## Constants

In [206]:
RATE=44100

## Utilities

In [7]:
class Note(Enum):
    C3 = 131
    D3 = 147
    E3 = 165
    F3 = 175
    G3 = 196
    A4 = 220
    B4 = 247
    C4 = 262

In [8]:
def inspect_sound(sound_data):
    df = pd.DataFrame(sound_data)
    print("shape: ", df.shape)
    print("head: ")
    df.head(101).plot()
    stream.write(df.values)

## Signal Generators

In [9]:
def wave(frequency=Note.C3.value, length=1, amplitude=5000, phase=0, sample_rate=RATE, function=None, function_args=None):
    time = linspace(0,length,length*sample_rate)
    wavelength = 1/frequency
    data = function(time, wavelength, amplitude, phase, function_args)
    return data.astype(int16) # two byte integers

In [10]:
def sine_function(time, wavelength, amplitude, phase, function_args=None):
    return amplitude * sin((2*pi*time - phase)/wavelength)

In [11]:
def square_function(time, wavelength, amplitude, phase, function_args=None):
    return amplitude * sign(sin((2*pi*time - phase)/wavelength))

In [12]:
def triangle_function(time, wavelength, amplitude, phase, function_args=None):
    return (2*amplitude/pi) * arcsin(sin((2*pi*time - phase)/wavelength))

In [13]:
def sawtooth_function(time, wavelength, amplitude, phase, function_args=None):
    return (2*amplitude/pi) * arctan(tan((2*pi*time - phase)/(2*wavelength)))

In [14]:
def pulse_function(time, wavelength, amplitude, phase, function_args):
    pulse_width = function_args['pulse_width']
    return amplitude * sign(sin((2*pi*time - phase)/wavelength)-(1-pulse_width))

In [15]:
def white_noise_function(time, wavelength, amplitude, phase, function_args):
    return random.random(len(time))*amplitude

In [210]:
def silence(time, wavelength, amplitude, phase, function_args):
    return zeros(len(time))

In [170]:
def identity(signal):
    return signal

In [22]:
def frequency_modulation_synthesizer(carrier=Note.C3.value, length=1, amplitude=5000, sample_rate=RATE, modulation_depth=0, modulation_frequency=0):
    time = linspace(0,length,length*sample_rate)
    return (amplitude*sin(2*pi*time*carrier + modulation_depth*sin(2*pi*time*modulation_frequency))).astype(int16)

## Plugins

In [16]:
def linear_asdr_envelope(sound, attack, decay, sustain, release):
    peak = sound.max()
    a = linspace(0,1,RATE*attack)
    d = linspace(1,sustain,RATE*decay)
    s = linspace(sustain,sustain,len(sound)-(RATE*(attack+decay+release)))
    r = linspace(sustain,0,RATE*release)
    envelope = append(append(a,d),append(s,r))
    return multiply(envelope,sound).astype(int16)

In [17]:
def time_warp(sound, coefficient):
    if coefficient >= 1:
        return repeat(sound, coefficient)
    else:
        slicer = int(1 / coefficient)
        return sound[0::slicer].astype(int16)

In [18]:
def lfo(sound, frequency, amount=1, wave_function=sine_function, wave_function_args={}):
    lfo = wave(function=wave_function, frequency=frequency, length=int(len(sound)/RATE), function_args=wave_function_args)
    lfo = add(lfo, lfo.max()) # shift lfo to positive
    lfo = divide(lfo,lfo.max()) # Make lfo range 0 to 1
    lfo = multiply(lfo,amount) # scale lfo by amount
    lfo = add(lfo, (1-amount)) # shift lfo up so peak is at 1 and min is (1-amount)
    return multiply(lfo,sound).astype(int16)

In [171]:
def arpeggiator(sound, step_size, frequency):
    return lfo(sound, frequency=frequency, wave_function=pulse_function, wave_function_args={'pulse_width':step_size})

In [20]:
def distortion(sound, amount):
    sound = multiply(sound, amount)
    amount = 1 / amount
    return clip(sound, int(sound.min()*amount), int(sound.max()*amount)).astype(int16)

In [21]:
def utility(sound, amount):
    return multiply(sound, amount).astype(int16)

In [196]:
def limiter(signal, amount):
    signs = sign(signal)
    return multiply(minimum(abs(signal),signal.max() * amount), signs).astype(int16)

In [23]:
def offset(sound, length=0):
    shift_amount = int(min(abs(length*RATE), len(sound)))
    shifted = zeros(sound.shape)
    if length >=0:
        shifted[shift_amount:] = sound[0:len(sound)-shift_amount]
    else:
        shifted[0:shift_amount] = sound[len(sound) - shift_amount:]
    return shifted.astype(int16)

In [25]:
def moving_average_low_pass_filter(sound, periods):
    max_amplitude = sound.max()
    cumulative = cumsum(sound)
    cumulative[periods:] = cumulative[periods:] - cumulative[:-periods]
    unadjusted = cumulative[periods - 1:] / periods
    louder = (max_amplitude / unadjusted.max()) * unadjusted
    return (louder).astype(int16)

In [133]:
def stft_low_pass_filter(sound, cutoff, amount=0):
    f, t, Zxx = stft(sound)
    for i,x in enumerate(Zxx.real):
        if i > cutoff:
            Zxx[i] = multiply(Zxx[i],zeros(x.shape)+amount)
    return istft(Zxx)[1].astype(int16)

In [135]:
def stft_high_pass_filter(sound, cutoff, amount=0):
    f, t, Zxx = stft(sound)
    for i,x in enumerate(Zxx.real):
        if i < cutoff:
            Zxx[i] = multiply(Zxx[i],zeros(x.shape)+amount)
    return istft(Zxx)[1].astype(int16)

In [136]:
def stft_band_pass_filter(sound, cutoff_lo, cutoff_hi, amount=0):
    f, t, Zxx = stft(sound)
    for i,x in enumerate(Zxx.real):
        if i < cutoff_lo or i > cutoff_hi:
            Zxx[i] = multiply(Zxx[i],zeros(x.shape)+amount)
    return istft(Zxx)[1].astype(int16)

## Workspace

In [207]:
stream = p.open(output=True, channels=2, rate=RATE, format=pyaudio.paInt16, output_device_index=1)

In [208]:
sig = frequency_modulation_synthesizer(length=1, modulation_depth=15, modulation_frequency=Note.C3.value+15)
stream.write(limiter(sig,0.5))

In [211]:
saw_c = wave(function=sawtooth_function, length=1, frequency=Note.C3.value)
saw_e = wave(function=sawtooth_function, length=1, frequency=Note.E3.value)
saw_g = wave(function=sawtooth_function, length=1, frequency=Note.G3.value)
saw_b = wave(function=sawtooth_function, length=1, frequency=Note.B4.value)
rest = wave(function=silence, length=1)

stream.write(saw_c)
stream.write(saw_e)
stream.write(saw_g)
stream.write(saw_c)
stream.write(saw_e)
stream.write(saw_g)
stream.write(saw_c)
stream.write(saw_e)
stream.write(saw_g)
stream.write(saw_c)
stream.write(saw_e)
stream.write(saw_g)
stream.write(rest)
stream.write(saw_e)
stream.write(saw_g)
stream.write(saw_b)
stream.write(saw_e)
stream.write(saw_g)
stream.write(saw_b)
stream.write(saw_e)
stream.write(saw_g)
stream.write(saw_b)
stream.write(saw_e)
stream.write(saw_g)
stream.write(saw_b)

#### -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Notes

### TODO
- LFO (works on sounds but not samples?)
- ADSR envelope
- Filters (FFT?)
- Arp (very simplistic right now.  Add a function to it for stepping up each note.  Pass Identity fun by default)
- Sampler
- How to change BPM?
    - should it be during playback like changing the stream rate
    - can I double every value in the array to make twice as long?
- distortion
- limiter
- "utility"


- Oscilator (wav combiners) or combining of synths
- reverb
- delay
- style transfer of samples (vocoder?)

### HOW TO USE
stream.write(wave(function=sine_function))

stream.write(wave(function=square_function))

stream.write(wave(function=triangle_function))

stream.write(wave(function=sawtooth_function))

stream.write(wave(function=pulse_function, function_args={'pulse_width':0.25}))

stream.write(wave(function=white_noise_function))

stream.write(linear_asdr_envelope(wave(function=square_function, length=6), 1, 1, 0.5, 1))

stream.write(wave(function=sine_function, length=3, frequency=Note.C3.value, amplitude=5000)+wave(function=sine_function, length=3, frequency=Note.C3.value*2, amplitude=2000))

stream.write(lfo(wave(function=sawtooth_function,length=9),frequency=3, amount=1))

stream.write(distortion(wave(function=sawtooth_function), 10))

stream.write(utility(wave(function=sawtooth_function, length=3), 2))

stream.write(frequency_modulation_synthesizer(length=4, modulation_depth=20, modulation_frequency=Note.C3.value))

stream.write(offest(wave(function=sine_function),0.5))

stft_low_pass_filter(signal,3,0.25)

limiter(sig,0.5)

##### Sampler
path = "D:\\Pat\\projects\\programming\\Accent Removal\\Samples\\wavs\\samplewav.wav"

stream.write(wavfile.read(path)[1])